In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.fpm import FPGrowth

In [ ]:
# data.csv uploaded as table named 'data' in databricks
data = spark.sql('SELECT* FROM data')
data.show(5)
data.registerTempTable("data")

In [ ]:
#concatenate prefix to variables to create unqiue itemsets
data_str = sqlContext.sql("SELECT CONCAT ('M', Month) AS Month, CONCAT ('W', Weekday) AS Weekday, CONCAT ('T', Time_period) AS Time_period,CONCAT ('F', Fare) AS Fare, CONCAT ('P', Pickup) AS Pickup, CONCAT ('D', Dropoff) AS Dropoff FROM data")
data_str.show(5)
data_str.registerTempTable("df")

In [ ]:
# generate arraylist as required by FPGrowth of ml package
df_concat = sqlContext.sql("SELECT CONCAT(Month, ','  ,Weekday, ','  ,Time_period, ',', Pickup, ',', Dropoff, ',', Fare) AS Features FROM df ")
df_array = df_concat.withColumn(
    "items", split(col("features"), ",\s*").cast(ArrayType(StringType())).alias("items"))
df_array=df_array.select("items")
df_array.show(5, False)

# Min Support = 0.001, Min Confidence = 0.6

In [ ]:
fp = FPGrowth(minSupport=0.001, minConfidence=0.6)
fpm = fp.fit(df_array) 

In [ ]:
#Frequent location Mining
freq = fpm.freqItemsets
freq.registerTempTable("df")
freq_result = sqlContext.sql("SELECT * FROM df ORDER BY freq DESC")
display(freq_result)

In [ ]:
#Association Rule mining
rules = fpm.associationRules
rules = rules.withColumn("consequent", concat_ws("",col("consequent")))
rules.show(5)
rules.registerTempTable("df")
rules_result = sqlContext.sql("SELECT * FROM df WHERE consequent='F2' ORDER BY confidence DESC")
display(rules_result)

# Association Rule without dominant locations

# Min Support = 0.0005, Min Confidence = 0.6


In [ ]:
# Remove Ariports: O'Hare and Midway 
data_str.registerTempTable("df")
noap = sqlContext.sql("SELECT * FROM df WHERE Pickup NOT IN ('P225006','P433757','P575547','P230394') AND Dropoff NOT IN ('D225006','D433757','D575547','D230394')")
display(noap)
noap.registerTempTable("df")

In [ ]:
df_concat = sqlContext.sql("SELECT CONCAT(Month, ','  ,Weekday, ','  ,Time_period, ',', Pickup, ',', Dropoff, ',', Fare) AS Features FROM df ")
df_concat.show(5, False)

In [ ]:
fp = FPGrowth(minSupport=0.0005, minConfidence=0.6)
fpm = fp.fit(df_array) 

In [ ]:
#Frequent location Mining
freq = fpm.freqItemsets
freq.registerTempTable("df")
freq_result = sqlContext.sql("SELECT * FROM df ORDER BY freq DESC")

In [ ]:
#Association Rule mining
rules = fpm.associationRules
rules = rules.withColumn("consequent", concat_ws("",col("consequent")))
rules.show(5)
rules.registerTempTable("df")
rules_result = sqlContext.sql("SELECT * FROM df WHERE consequent='F2' ORDER BY confidence DESC")
display(rules_result)